## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
#Importing relevant packages 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

import scipy.stats as stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.stats.diagnostic import het_breuschpagan
import sklearn.metrics as metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.stattools import jarque_bera
from scipy.stats import kstest

#Importing to use if needed
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D

import warnings


In [4]:
# Importing Dataset 
kc_df = pd.read_csv("data/kc_house_data.csv")
kc_df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,...,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052
1,8910500230,12/13/2021,920000.0,5,2.5,2770,6703,1.0,NO,NO,...,PUBLIC,1570,1570,0,240,1950,0,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.35591
2,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,...,PUBLIC,1580,1580,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.22520
3,1604601802,12/14/2021,775000.0,3,3.0,2160,1400,2.0,NO,NO,...,PUBLIC,1090,1070,200,270,2010,0,"4079 Letitia Avenue South, Seattle, Washington...",47.566110,-122.29020
4,8562780790,8/24/2021,592500.0,2,2.0,1120,758,2.0,NO,NO,...,PUBLIC,1120,550,550,30,2012,0,"2193 Northwest Talus Drive, Issaquah, Washingt...",47.532470,-122.07188


In [8]:
# Using .shape to check for number of rows and columns
print(kc_df.shape)

# Using .info() to check out columns, datatypes and to check for missing values 
print(kc_df.info())

# Checking value_counts of variables 
for col in kc_df:
    print(kc_df[col].value_counts(), "\n")

(30155, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30155 entries, 0 to 30154
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             30155 non-null  int64  
 1   date           30155 non-null  object 
 2   price          30155 non-null  float64
 3   bedrooms       30155 non-null  int64  
 4   bathrooms      30155 non-null  float64
 5   sqft_living    30155 non-null  int64  
 6   sqft_lot       30155 non-null  int64  
 7   floors         30155 non-null  float64
 8   waterfront     30155 non-null  object 
 9   greenbelt      30155 non-null  object 
 10  nuisance       30155 non-null  object 
 11  view           30155 non-null  object 
 12  condition      30155 non-null  object 
 13  grade          30155 non-null  object 
 14  heat_source    30123 non-null  object 
 15  sewer_system   30141 non-null  object 
 16  sqft_above     30155 non-null  int64  
 17  sqft_basement  30155 non-null  int64  

The dataset contains 30155 rows with 25 columns. Datatypes for the columns include integers, floats and objects. 

- The following columns have missing values: heat_source (33) and sewer_system (14). Due to the low amount of missing values, these rows will be dropped as these make up a very minimal percentage of the dataset. 

- Ten columns have their datatypes as objects while 15 are numeric.

- The grade column mixes two datatypes in which it has a numeric value(formatted as a string) and an object (as a string).

- Waterfront, greenbelt, nuisance, view, condition, heat_source, and sewer_system all have categorical data.

- The view column has five different categories: NONE, AVERAGE, GOOD, EXCELLENT, FAIR. In this column, it seems 26,589 rows (88.2% of the dataset) are categorized as NONE. 

- Sqft_basement has 15,471 entries with a value of 0. This is roughly 51.3% of the dataset. It may be useful to change this variable to a categorical variable indicating if there is a basement or not (YES/NO). YES if the value is greater than 0 and NO if the value is 0. 

- Sqft_garage and sqft_patio are similar to to sqft_basement and could be simplified to a catergorical variable utilizing YES/NO. 

In [5]:
# Checking general statistics of dataset 
kc_df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,lat,long
count,3.015500e+04,3.015500e+04,30155.000000,30155.000000,30155.000000,3.015500e+04,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000,30155.000000
mean,4.538104e+09,1.108536e+06,3.413530,2.334737,2112.424739,1.672360e+04,1.543492,1809.826098,476.039396,330.211142,217.412038,1975.163953,90.922301,47.328076,-121.317397
std,2.882587e+09,8.963857e+05,0.981612,0.889556,974.044318,6.038260e+04,0.567717,878.306131,579.631302,285.770536,245.302792,32.067362,416.473038,1.434005,5.725475
min,1.000055e+06,2.736000e+04,0.000000,0.000000,3.000000,4.020000e+02,1.000000,2.000000,0.000000,0.000000,0.000000,1900.000000,0.000000,21.274240,-157.791480
25%,2.064175e+09,6.480000e+05,3.000000,2.000000,1420.000000,4.850000e+03,1.000000,1180.000000,0.000000,0.000000,40.000000,1953.000000,0.000000,47.405320,-122.326045
50%,3.874011e+09,8.600000e+05,3.000000,2.500000,1920.000000,7.480000e+03,1.500000,1560.000000,0.000000,400.000000,150.000000,1977.000000,0.000000,47.551380,-122.225585
75%,7.287100e+09,1.300000e+06,4.000000,3.000000,2619.500000,1.057900e+04,2.000000,2270.000000,940.000000,510.000000,320.000000,2003.000000,0.000000,47.669913,-122.116205
max,9.904000e+09,3.075000e+07,13.000000,10.500000,15360.000000,3.253932e+06,4.000000,12660.000000,8020.000000,3580.000000,4370.000000,2022.000000,2022.000000,64.824070,-70.074340


Looking over the initial general statistics here are a few quick insights into the data:

- The average price of a house in King county is approximately 1,108,536 USD with the median price being 860,000 USD, the maximum price being 30,750,000 USD and the minimum price being 27,360 USD. The standard deviation of the price is 896,385.70 USD.

- The average square foot of living space (sqft_living) of a house in King Count is approximately 2112.42 square feet with the median being 1920.00 square feet, the maximum being 15,360 square feet, the minimum being 3 square feet and the standard deviation being 974.044. The minimum being 3 square feet does not seem like it would be correct and may be an error. This warrants more investigation into that lower value.

- The average number of bedrooms of a house in King County is 3.41 bedrooms, with the median number of bedrooms being 3, the maximum number of bedrooms being 13 and the minimum number being 0. The standard deviation is 0.982. One thing to note is that the minimum number is 0 which shows that there are houses with no bedrooms. Looking at the value_counts above we see that 44 houses have 0 bedrooms. We would expect every house to have at least 1 bedroom. With 44 being a very small percentage of the dataset we could drop these rows. 

- The average number of bathrooms of a house in King County is 2.33 bathrooms, with the median number of bathrooms being 2.5, the maximum number of bathrooms being 10.5 and the minimum number being 0. The standard deviation is 0.90. Similar to the number of bedrooms, the minimum number of bathrooms is 0. Looking at the value_counts above we see that 36 houses have less than 1 (either 0 or 0.5) bathrooms.  We would expect every house to have at least 1 full bathroom. With 36 being a very small percentage of the dataset we could drop these rows.  

- The average number of floors of a house in King County is 1.54 florr, with the median number being 1.5, the maximum number being 4 and the minimum number being 1. The standard deviation is 0.568. 